## Advanced functionality
The examples above should cover more than 90% of the scenarios that you are are likely to use argparse for. It has some rather powerful additional features, however, and in the following sections we’ll see how it can be used to perform more specific input validation as well as managing input and output streams.

### Parsing custom data
As well as using the predefined types such as str, int, float etc, the type= parameter can accept any callable that takes a single string argument and returns a value. This can be used to do custom validation or conversion of the input:

In [ ]:
import argparse

parser = argparse.ArgumentParser()

def single_word(string):
    # Check input does not contain spaces
    if (' ' in string):
        msg = f'\"{string}\" is not a single word'
        raise argparse.ArgumentTypeError(msg)
    return string

parser.add_argument('argument1', type=single_word)

In the example above, an exception is raised if the input contains one or more space characters:

```shell
$ python argparse_custom_type.py abc123
abc123

$ python argparse_custom_type.py "Hello, world!"
usage: argparse_custom_type.py [-h] argument1
error: argument1: "Hello, world!" is not a single word
```

### Opening and closing files

It is relatively common to use command line arguments to specify paths to input and output files, such as for source data and results summaries. `argparse` can handle that for you: ***FileType*** gives you a more flexible way of specifying that an argument should be a file, and can handle encoding, access mode (read/write/append etc) and other hyperparameters:

In [ ]:
import argparse

parser = argparse.ArgumentParser()

# Add a required, positional argument for the input data file name,
# and open in 'read' mode
parser.add_argument('infile', type=argparse.FileType('r'))

# Add an optional argument for the output file,
# open in 'write' mode and and specify encoding
parser.add_argument('--output', type=argparse.FileType('w', encoding='UTF-8'))

args = parser.parse_args()

# Read a CSV file,  sum the values in the second column,
# and optionally write to file
sum = 0
with args.infile as infile:
    for line in infile:
        value = int(line.split(',')[1])
        sum += value
        print(sum)
    
        if args.output is not None:
            args.output.writelines(f'{sum}\n')

When using the FileType type, argparse takes care of opening the file(s) for you. If the input file name is not found, it throws an error that references the associated argument; if the optional output file argument is supplied, argparse creates the output stream:

```bash
$ python argparse_FileType.py source_dta.csv
usage: argparse_FileType.py [-h] [--output OUTPUT] infile
argparse_FileType.py.py: error: argument infile: can't open 'source_dta.csv': [Errno 2] No such file or directory: 'source_dta.csv'
$ python basic.py source_dta.csv --output sum.csv
```